In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from find_screen_utils import *

#Add Gaze Position to Video

In [13]:
gaze = pd.read_csv('images/tobii/border.csv')
vid_sync = pd.read_csv('images/tobii/border_video_sync.csv')

In [15]:
vid_sync.head()

,Unnamed: 0,VTS time,VTS validity
0,89956352,0,0
1,90316397,359888,0
2,149941025,59985688,0
3,209927788,119971377,0
4,269912195,179956966,0


In [16]:
gaze['Vid_Time'] = (gaze['index'] - 89956352)/1000000

In [18]:
gaze = gaze[gaze.Vid_Time > 0] # only start tracking eyes once video starts

In [20]:
infile = 'images/tobii/border.mp4'
outfile = 'images/tobii/border_gaze.m4v'

In [21]:
vid = cv2.VideoCapture(infile)

size = (1920, 1080)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS)*2, size, True)
# note doubled framerate to include all eye tracking coordiantes

i = 0
gaze_val = gaze['gaze position validity'].values
gaze_x = gaze['gaze position x'].values
gaze_y = gaze['gaze position y'].values

while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        for _ in range(2):
            frame_cp = frame.copy()
            if gaze_val[i] == 0:
                gp_x = gaze_x[i]
                gp_y = gaze_y[i]
                cv2.circle(frame_cp, (int(1920*gp_x), int(1080*gp_y)), 20, [255,0,0], 2)
            out.write(frame_cp)
            i += 1
    else:
        break


vid.release()
out.release()

#Extract Screen From Gaze Video

In [3]:
infile = 'images/tobii/border/border_gaze_dark_dots.mov'
outfile = 'images/tobii/border/border_gaze_dark_dots_screen2.m4v'

In [4]:
vid = cv2.VideoCapture(infile)

size = (1280, 720)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS), size, True)

last_good = np.zeros((720, 1280, 3), dtype=np.uint8)  # represents the last good frame

i = 0
while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        proc_frame = process_frame_circles(frame)
        
        if proc_frame is None:  # if screen isn't found, save image of frame
            out.write(last_good)  # write the last good frame to the video
#             cv2.imwrite('images/tobii/dropped/frame%i.bmp'%(i), frame)
        else:  # frame is good
            last_good = proc_frame
            out.write(proc_frame)
        i += 1
    else:
        break


vid.release()
out.release()